[Reference](https://medium.com/@dayooped/web-scraping-in-etl-process-976a96920efe)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
data = requests.get(wiki_url).text

In [3]:
soup = BeautifulSoup(data, "html.parser")

In [4]:
tables = soup.find_all("table")
len(tables)

70

In [5]:
for index, table in enumerate(tables):
    print(index, table)
# alternatively, to locate the table index using the table name
# for index,table in enumerate(tables):
#     if ("<insert_table_name>" in str(table)):
#         table_index = index
# print(table_index)

Streaming output truncated to the last 5000 lines.
<li><a href="/wiki/COVID-19_pandemic_in_Gibraltar" title="COVID-19 pandemic in Gibraltar">Gibraltar</a></li></ul></li></ul>
</div></td></tr></tbody></table><div></div></td></tr><tr><th class="navbox-group" scope="row" style="width:1%">Eastern</th><td class="navbox-list-with-group navbox-list navbox-even" style="width:100%;padding:0"><div style="padding:0 0.25em">
<ul><li><a href="/wiki/COVID-19_pandemic_in_Belarus" title="COVID-19 pandemic in Belarus">Belarus</a>
<ul><li>timeline
<ul><li><a href="/wiki/Timeline_of_the_COVID-19_pandemic_in_Belarus_(2020)" title="Timeline of the COVID-19 pandemic in Belarus (2020)">2020</a></li>
<li><a href="/wiki/Timeline_of_the_COVID-19_pandemic_in_Belarus_(2021)" title="Timeline of the COVID-19 pandemic in Belarus (2021)">2021</a></li>
<li><a href="/wiki/Timeline_of_the_COVID-19_pandemic_in_Belarus_(2022)" title="Timeline of the COVID-19 pandemic in Belarus (2022)">2022</a></li></ul></li></ul></li>
<l

In [11]:
covid_data = pd.read_html(str(tables[12]), flavor="bs4")[0]
covid_data.head() # To print the first 5 rows 
covid_data.tail() # To print the last 5 rows

,Unnamed: 0,Country,Deaths / million,Deaths,Cases
216,NaN,Falkland Islands,—,—,1930
217,NaN,Vatican City,—,0,29
218,NaN,"Saint Helena, Ascension and Tristan da Cunha",—,—,2013
219,NaN,Tuvalu,—,—,20
220,.mw-parser-output .reflist{font-size:90%;margi...,.mw-parser-output .reflist{font-size:90%;margi...,.mw-parser-output .reflist{font-size:90%;margi...,.mw-parser-output .reflist{font-size:90%;margi...,.mw-parser-output .reflist{font-size:90%;margi...


In [13]:
covid_data = covid_data.drop("Unnamed: 0", axis=1)
covid_data.head()

,Country,Deaths / million,Deaths,Cases
0,World[a],831,6577251,627195515
1,Peru,6432,216877,4150737
2,Bulgaria,5492,37820,1273949
3,Bosnia and Herzegovina,4942,16165,399663
4,Hungary,4922,47798,2132490


In [14]:
# remove the last row
covid_data = covid_data.drop(covid_data.index[[220]])
#remove the "-" in "Deaths / million" column
covid_data.drop(covid_data.index[(covid_data["Deaths / million"] == "—")],axis=0, inplace=True)
# print out the data
print(covid_data)

                    Country Deaths / million   Deaths      Cases
0                  World[a]              831  6577251  627195515
1                      Peru             6432   216877    4150737
2                  Bulgaria             5492    37820    1273949
3    Bosnia and Herzegovina             4942    16165     399663
4                   Hungary             4922    47798    2132490
..                      ...              ...      ...        ...
211                    Chad               11      193       7613
212                   Macau                8        6        793
213                China[c]                3     5226    1018065
214                 Burundi                3       38      50413
215             North Korea                0        6          1

[216 rows x 4 columns]


In [15]:
# Rename the value
covid_data["Country"][0] = "World"
covid_data["Country"][34] = "European Union"
covid_data["Country"][213] = "China"

In [16]:
# Convert data into numeric and insert comma as a thousands separator
covid_data["Deaths / million"] = covid_data["Deaths / million"].apply(pd.to_numeric).apply(lambda x : "{:,}".format(x))
covid_data["Deaths"] = covid_data["Deaths"].apply(pd.to_numeric).apply(lambda x : "{:,}".format(x))
covid_data["Cases"] = covid_data["Cases"].apply(pd.to_numeric).apply(lambda x : "{:,}".format(x))
covid_data.head()

,Country,Deaths / million,Deaths,Cases
0,World,831,"6,577,251","627,195,515"
1,Peru,"6,432","216,877","4,150,737"
2,Bulgaria,"5,492","37,820","1,273,949"
3,Bosnia and Herzegovina,"4,942","16,165","399,663"
4,Hungary,"4,922","47,798","2,132,490"


In [19]:
covid_data.to_csv("covid_data.csv")
df = pd.read_csv("covid_data.csv")
print(df)

     Unnamed: 0                 Country Deaths / million     Deaths  \
0             0                   World              831  6,577,251   
1             1                    Peru            6,432    216,877   
2             2                Bulgaria            5,492     37,820   
3             3  Bosnia and Herzegovina            4,942     16,165   
4             4                 Hungary            4,922     47,798   
..          ...                     ...              ...        ...   
211         211                    Chad               11        193   
212         212                   Macau                8          6   
213         213                   China                3      5,226   
214         214                 Burundi                3         38   
215         215             North Korea                0          6   

           Cases  
0    627,195,515  
1      4,150,737  
2      1,273,949  
3        399,663  
4      2,132,490  
..           ...  
211        7,6